In [0]:
# call the heavenly gods
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, bias=False, padding=1) #input 28x28x1, output 28x28x8, RF 3,
        self.bn1 = nn.BatchNorm2d(8)
        self.d1 = nn.Dropout(0.05) #5 % probability 

        self.conv2 = nn.Conv2d(8, 8, 3, bias=False, padding=1) #input 28x28x8, output 28x28x8, RF 5
        self.bn2 = nn.BatchNorm2d(8)
        self.d2 = nn.Dropout(0.05) #5 % probability 

        self.pool1 = nn.MaxPool2d(2, 2) # input 28x28x8 output 14x14x8 RF 6

        self.conv3 = nn.Conv2d(8, 12, 3, bias=False, padding=1) #input 14x14x8, output 14x14x8, RF 10
        self.bn3 = nn.BatchNorm2d(12)
        self.d3 = nn.Dropout(0.05) #5 % probability 

        self.conv4 = nn.Conv2d(12, 12, 3, bias=False, padding=1) #input 14x14x8, output 14x14x8, RF 14
        self.bn4 = nn.BatchNorm2d(12)
        self.d4 = nn.Dropout(0.05) #5 % probability

        self.pool2 = nn.MaxPool2d(2, 2) # input 14x14x8 output 7x7x8 RF 10

        self.conv5 = nn.Conv2d(12, 16, 3, bias=False, padding=1) #input 7x7x8, output 7x7x16, RF 14
        self.bn5 = nn.BatchNorm2d(16)
        self.d5 = nn.Dropout(0.05) #5 % probability 
        
        self.conv6 = nn.Conv2d(16, 16, 3, bias=False) #input 7x7x16, output 5x5x16, RF 18
        self.bn6 = nn.BatchNorm2d(16)
        self.d6 = nn.Dropout(0.05) #5 % probability 

        # self.conv7 = nn.Conv2d(16, 16, 3, bias=False) #input 6x6x16, output 4x4x16, RF 22
        # self.bn7 = nn.BatchNorm2d(16)
        # self.d7 = nn.Dropout(0.05) #5 % probability 

        # self.conv8 = nn.Conv2d(16, 16, 3, bias=False) #input 4x4x16, output 2x2x16, RF 26
        # self.bn8 = nn.BatchNorm2d(16)
        # self.d8 = nn.Dropout(0.05) #5 % probability 
        
        self.conv7 = nn.Conv2d(16, 10, 1, bias=False) #input 2x2x16, output 2x2x10, RF 26

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)

        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.conv7(x)

        x = F.avg_pool2d(x, 5)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [62]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3            [-1, 8, 28, 28]             576
       BatchNorm2d-4            [-1, 8, 28, 28]              16
         MaxPool2d-5            [-1, 8, 14, 14]               0
            Conv2d-6           [-1, 12, 14, 14]             864
       BatchNorm2d-7           [-1, 12, 14, 14]              24
            Conv2d-8           [-1, 12, 14, 14]           1,296
       BatchNorm2d-9           [-1, 12, 14, 14]              24
        MaxPool2d-10             [-1, 12, 7, 7]               0
           Conv2d-11             [-1, 16, 7, 7]           1,728
      BatchNorm2d-12             [-1, 16, 7, 7]              32
           Conv2d-13             [-1, 16, 5, 5]           2,304
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    correct = 0
    pbar =  tqdm_notebook(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

    train_loss /= len(train_loader.dataset)

    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [65]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in tnrange(1, 21):
    print("\nEpoch:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train set: Average loss: 0.6391, Accuracy: 50161/60000 (83.60%)
Test set: Average loss: 0.1848, Accuracy: 9501/10000 (95.01%)

Epoch: 2


Train set: Average loss: 0.0918, Accuracy: 58653/60000 (97.75%)
Test set: Average loss: 0.1353, Accuracy: 9593/10000 (95.93%)

Epoch: 3


Train set: Average loss: 0.0598, Accuracy: 59071/60000 (98.45%)
Test set: Average loss: 0.0672, Accuracy: 9797/10000 (97.97%)

Epoch: 4


...

Train set: Average loss: 0.0176, Accuracy: 59693/60000 (99.49%)
Test set: Average loss: 0.0267, Accuracy: 9919/10000 (99.19%)

Epoch: 17


Train set: Average loss: 0.0166, Accuracy: 59711/60000 (99.52%)
Test set: Average loss: 0.0243, Accuracy: 9920/10000 (99.20%)

Epoch: 18


Train set: Average loss: 0.0158, Accuracy: 59742/60000 (99.57%)
Test set: Average loss: 0.0228, Accuracy: 9921/10000 (99.21%)

Epoch: 19


Train set: Average loss: 0.0144, Accuracy: 59758/60000 (99.60%)
Test set: Average loss: 0.0234, Accuracy: 9920/10000 (99.20%)

Epoch: 20


Train set: Average loss: 0.0135, Accuracy: 59778/60000 (99.63%)
Test set: Average loss: 0.0219, Accuracy: 9916/10000 (99.16%)
